In [124]:
#import sys
#sys.argv.extend(['-J-Xmx4G'])  

import os
import esa_snappy
from esa_snappy import GPF
from esa_snappy import ProductIO, GeoPos, PixelPos, WKTReader
from esa_snappy import HashMap
from esa_snappy import jpy
import subprocess
from time import *
import numpy as np
from shapely.geometry import Point, Polygon
import geopandas as gpd

HashMap = jpy.get_type('java.util.HashMap')
#parameters = HashMap()

Inputs have to be:
- .SAFE folders
- SLC data
- IW beam mode

# **Reading and getting information**

In [125]:
a = '_data\S1A_IW_SLC__1SDV_20250519T184325_20250519T184352_059267_075AD4_7DCB.SAFE' #iceland 2025
b = '_data\S1C_IW_SLC__1SDV_20250520T183354_20250520T183421_002418_005114_74D2.SAFE'

#c = '_data\S1B_IW_SLC__1SDV_20190519T074026_20190519T074056_016310_01EB23_6796.SAFE' #iceland 2019
#d = '_data\S1A_IW_SLC__1SDV_20190515T184306_20190515T184332_027242_03124F_138F.SAFE'

e = '_data\S1A_IW_SLC__1SDV_20211219T191349_20211219T191419_041082_04E179_123E.SAFE' #spain
f = '_data\S1B_IW_SLC__1SDV_20210908T191308_20210908T191338_028611_036A16_4F4E.SAFE'

In [126]:
def extract_info(product_path):
    product = read_product(product_path)
    print("Product name:", product.getName())
    print("Product type:", product.getProductType())
    print("Description:", product.getDescription())
    #print("Beam Mode:", check_beam_mode(product_path))
    print("Scene width:", product.getSceneRasterWidth())
    print("Scene height:", product.getSceneRasterHeight())

    metadata = product.getMetadataRoot()
    print("Start time:", product.getStartTime())
    print("End time:", product.getEndTime())

    print("\n Bands")
    count = 0
    band_names = product.getBandNames()
    number_bands=len(list(band_names))
    print("Number of bands:", number_bands)
    while count<number_bands:
        band = product.getBand(band_names[count])
        print("Band name:", band.getName())
        count = count + 1   
    product.dispose()
    print("\n")
    return

In [127]:
#extract_info(a)
#extract_info(b)

#extract_info(c)
#extract_info(d)

#extract_info(e)
#extract_info(f)

In [128]:
def read_product(product_path):
    product = ProductIO.readProduct(product_path)
    return product

In [134]:
#read_product(e)

In [135]:
master1 = read_product(a)
slave1 = read_product(b)

#master2 = read_product(c)
#slave2 = read_product(d)

master3 = read_product(f)
slave3 = read_product(e)

In [136]:
def write(product, save_path, extension):
    print("Writing the product...")
    #GeoTIFF = .tif
    #BEAM-DIMAP = .dim
    ProductIO.writeProduct(product, save_path, extension)
    print("Product written")

In [137]:
#write(master1, '_results\something_test', 'BEAM-DIMAP')

In [138]:
def temporal_baseline(product1, product2):
    master_time = product1.getStartTime()
    slave_time = product2.getStartTime()
    temporal_baseline = abs(slave_time.getMJD() - master_time.getMJD())

    print(f"Temporal Baseline: {temporal_baseline:.1f} days")

    product1.dispose()
    product2.dispose()
    return

In [139]:
#temporal_baseline(master1, slave1)
#temporal_baseline(master2, slave2)
#temporal_baseline(master3, slave3)

In [140]:
def get_subswath(aoi, product):
    if isinstance(aoi, str) and aoi.startswith('POLYGON'):
        coords_str = aoi.replace('POLYGON((', '').replace('))', '')
        coord_pairs = coords_str.split(',')
        coords = []
        for pair in coord_pairs:
            lon, lat = map(float, pair.strip().split())
            coords.append((lon, lat))
        aoi_polygon = Polygon(coords)
    elif isinstance(aoi, Polygon):
        aoi_polygon = aoi
    elif isinstance(aoi, (list, tuple)):
        aoi_polygon = Polygon(aoi)
    else:
        raise ValueError("AOI must be a WKT string, Shapely Polygon, or list of (lon, lat) tuples")
    
    centroid = aoi_polygon.centroid
    
    band_names = list(product.getBandNames())
    subswaths = set()
    
    for band_name in band_names:
        if '_IW' in band_name:
            sw_part = band_name.split('_IW')[1][:1]
            if sw_part.isdigit():
                subswaths.add(f"IW{sw_part}")
                
    result = "No subswath for the AOI"
    for subswath in sorted(subswaths):
        subswath_bands = [band for band in band_names if f'_IW{subswath[-1]}_' in band]
        if not subswath_bands:
            continue
            
        band = product.getBand(subswath_bands[0])
        geo_coding = band.getGeoCoding()
        
        if geo_coding:
            pixel_pos = geo_coding.getPixelPos(esa_snappy.GeoPos(centroid.y, centroid.x), None)
            width = band.getRasterWidth()
            height = band.getRasterHeight()
            
            if 0 <= pixel_pos.x < width and 0 <= pixel_pos.y < height:
                result = subswath
    
    return result

In [141]:
#aoi = [(-16.7502, 66.2085), (-15.5424, 66.2085), (-15.5424, 66.5664), (-16.7502, 66.5664), (-16.7502, 66.2085)]  #a certa
#aoi = [(-18.3927, 66.0324), (-17.939, 66.0324), (-17.939, 66.1997), (-18.3927, 66.1997), (-18.3927, 66.0324)]
#aoi = [(-24.6537, 65.4276), (-24.3714, 65.4276), (-24.3714, 65.528), (-24.6537, 65.528), (-24.6537, 65.4276)] #super fora
#POLYGON((-16.6667 66.1961,-15.6057 66.1961,-15.6057 66.5597,-16.6667 66.5597,-16.6667 66.1961)) # mesma peninsula mas para dados com mesmo subswath
aoi = [(-18.0631, 28.4088), (-17.6148, 28.4088), (-17.6148, 28.8836), (-18.0631, 28.8836), (-18.0631, 28.4088)] #spain
#get_subswath(aoi, master3)
#get_subswath(aoi, slave1)
get_subswath(aoi, slave3)

'IW2'

In [142]:
def check_same_subswath(product1, product2, aoi):
    subswath1 = get_subswath(aoi, product1)
    subswath2 = get_subswath(aoi, product2)
    if subswath1 == subswath2:
        print("Both product use the same subswath:", get_subswath(aoi, product1))
    else:
        print("For product", product1.getName(), "you'll need also to use to use the", get_subswath(aoi, product2), "subswath")
        print("For product", product2.getName(), "you'll need also to use to use the", get_subswath(aoi, product1), "subswath")

In [143]:
check_same_subswath(slave3, master3, aoi)

Both product use the same subswath: IW2


In [144]:
def topsar_merge(products, polarizations='VV'):
    parameters = HashMap()
    print('Merging...')
    parameters.put("selectedPolarisations", polarizations)
    if isinstance(products, list):
        merged_product = GPF.createProduct("TOPSAR-Merge", parameters, products)
    else:
        merged_product = GPF.createProduct("TOPSAR-Merge", parameters, products)
    
    return merged_product

In [145]:
#topsar_merge([topsar_deburst(topsar_split(master1, 'IW1', 'VH')), topsar_deburst(topsar_split(master1, 'IW2', 'VH'))], 'VH')

In [146]:
def merging_process(product1_to_merge, product2, polarization, aoi):
    subswath_product1 = get_subswath(aoi, product1_to_merge)
    subswath_product2 = get_subswath(aoi, product2)
    
    subswath1 = topsar_split(product1_to_merge,subswath_product1, polarization) 
    subswath2 = topsar_split(product1_to_merge,subswath_product2, polarization)

    deburst1 = topsar_deburst(subswath1)
    deburst2 = topsar_deburst(subswath2)

    merge = topsar_merge([deburst1, deburst2], polarization)

    return merge    

In [147]:
#merging_process(master1, slave1, 'VH', aoi)

In [148]:
def merge_all_subswath(product, polarization):    
    subswath1 = topsar_split(product, 'IW1', polarization) 
    subswath2 = topsar_split(product, 'IW2', polarization) 
    subswath3 = topsar_split(product, 'IW3', polarization) 

    deburst1 = topsar_deburst(subswath1)
    deburst2 = topsar_deburst(subswath2)
    deburst3 = topsar_deburst(subswath3)

    merge = topsar_merge([deburst1, deburst2, deburst3], polarization)

    return merge  

In [149]:
#merge_all_subswath(master1, 'VH')

# **Processing workflow starts**

In [150]:
def topsar_split(product, subswath, polarization='VV'):
    parameters = HashMap()
    print('Apply TOPSAR Split...')
    parameters.put('subswath', subswath)
    #parameters.put('firstBurstIndex', 1) 
    #parameters.put('lastBurstIndex', 8) 
    parameters.put('selectedPolarisations', polarization)
    output = GPF.createProduct("TOPSAR-Split", parameters, product)
    return output

In [151]:
split_master = topsar_split(master3, get_subswath(aoi, master3), 'VH')
split_slave = topsar_split(slave3, get_subswath(aoi, slave3), 'VH')
write(split_master, '_results\split_master', 'BEAM-DIMAP')
write(split_slave, '_results\split_slave', 'BEAM-DIMAP')

Apply TOPSAR Split...
Apply TOPSAR Split...
Writing the product...
Product written
Writing the product...
Product written


In [152]:
def apply_orbit_file(product):
    parameters = HashMap()
    print('Apply Orbit File...')
    parameters.put("Orbit State Vectors", "Sentinel Precise (Auto Download)")
    parameters.put("Polynomial Degree", 3)
    parameters.put("Do not fail if new orbit file is not found", True)
    output = GPF.createProduct("Apply-Orbit-File", parameters, product) 
    return output

In [153]:
orbit_master = apply_orbit_file(read_product('_results\split_master.dim'))
orbit_slave = apply_orbit_file(read_product('_results\split_slave.dim'))
write(orbit_master, '_results\orbit_master', 'BEAM-DIMAP')
write(orbit_slave, '_results\orbit_slave', 'BEAM-DIMAP')

Apply Orbit File...
Apply Orbit File...
Writing the product...
Product written
Writing the product...
Product written


In [154]:
def back_geocoding(products):
    parameters = HashMap()
    print('Back geocoding ...')
    parameters.put("Digital Elevation Model", "SRTM 1Sec HGT (Auto Download)")
    parameters.put("demResamplingMethod", "BILINEAR_INTERPOLATION")
    parameters.put("resamplingType", "BILINEAR_INTERPOLATION")
    parameters.put("maskOutAreaWithoutElevation", True)
    parameters.put('disableSpectralDiversity', True)
    parameters.put("outputDerampDemodPhase", True)
    parameters.put("disableReramp", False)
    #parameters.put("The list of source bands", "")
    output = GPF.createProduct("Back-Geocoding", parameters, products) 
    return output

In [155]:
#back_geocoding([merge_all_subswath(master1, 'VH'), merge_all_subswath(slave1, 'VH')])
back = back_geocoding([read_product(r'_results\orbit_master.dim'), read_product(r'_results\orbit_slave.dim')])
write(back, r'_results\back_geocoding', 'BEAM-DIMAP')

Back geocoding ...
Writing the product...
Product written


In [156]:
def enhanced_spectral_diversity(product):
    parameters = HashMap()
    print('Enhancing Spectral Diversity ...')
    parameters.put("fineWinWidthStr", "512")
    parameters.put("fineWinHeightStr", "512")
    parameters.put("fineWinAccAzimuth", "16")
    parameters.put("fineWinAccRange", "16")
    parameters.put("fineWinOversampling", "128")
    parameters.put("esdEstimator", "Periodogram")
    parameters.put("weightFunc", "Inv Quadratic")
    parameters.put("temporalBaselineType", "Number of images")
    parameters.put("integrationMethod", "L1 and L2")
    parameters.put("xCorrThreshold", 0.1)
    parameters.put("cohThreshold", 0.3)
    parameters.put("overallRangeShift", 0.0)
    parameters.put("overallAzimuthShift", 0.0)
    Integer = jpy.get_type('java.lang.Integer')
    parameters.put("numBlocksPerOverlap", Integer(10))
    parameters.put("maxTemporalBaseline", Integer(2))
    parameters.put("doNotWriteTargetBands", False)
    parameters.put("useSuppliedRangeShift", False)
    parameters.put("useSuppliedAzimuthShift", False)
    output = GPF.createProduct("Enhanced-Spectral-Diversity", parameters, product)
    return output

In [157]:
enhanced = enhanced_spectral_diversity(read_product(r'_results\back_geocoding.dim'))
write(enhanced, r'_results\enhanced', 'BEAM-DIMAP')

Enhancing Spectral Diversity ...
Writing the product...
Product written


In [160]:
def interferogram(product):
    parameters = HashMap()
    print('Creating interferogram ...')
    parameters.put("Subtract flat-earth phase", True)
    parameters.put("Degree of \"Flat Earth\" polynomial", 5)
    parameters.put("Number of \"Flat Earth\" estimation points", 501)
    parameters.put("Orbit interpolation degree", 3)
    parameters.put("Include coherence estimation", True)
    parameters.put("Square Pixel", True)
    parameters.put("Independent Window Sizes", False)
    #parameters.put("Coherence Azimuth Window Size", 10)
    #parameters.put("Coherence Range Window Size", 2)
    output = GPF.createProduct("Interferogram", parameters, product) 
    return output

In [161]:
interferogram = interferogram(read_product(r'_results\enhanced.dim'))
write(interferogram, r'_results\interferogram', 'BEAM-DIMAP')

Creating interferogram ...
Writing the product...
Product written


**maybe coherence is not neeed because on the interferogram we already compute it**

In [85]:
def coherence_estimation(product):
    parameters = HashMap()
    print('Coherence Estimation...')
    #parameters.put('cohWinAz', 2)          
    #parameters.put('cohWinRg', 10)        
    #parameters.put('subtractFlatEarthPhase', True)
    #parameters.put('srpPolynomialDegree', int(5))  # Explicit cast
    #parameters.put('srpNumberPoints', 501)
    #parameters.put('orbitDegree', 3)
    parameters.put('squarePixel', True)
    parameters.put('subtractTopographicPhase', False)
    #parameters.put('demName', 'SRTM 3Sec')
    #parameters.put('externalDEMNoDataValue', 0.0)
    #parameters.put('externalDEMApplyEGM', True)
    #parameters.put('tileExtensionPercent', 100)
    #parameters.put('singleMaster', False)
    output = GPF.createProduct('Coherence', parameters, product)
    return output

In [34]:
#coherence = coherence_estimation(interferogram)

In [162]:
def topsar_deburst(product, polarization='VV'):  
    parameters = HashMap()
    print('Apply TOPSAR Deburst...')
    parameters.put("Polarisations", polarization)
    output=GPF.createProduct("TOPSAR-Deburst", parameters, product)
    return output

In [163]:
deburst = topsar_deburst(read_product(r'_results\interferogram.dim'), 'VH')
write(deburst, r'_results\deburst', 'BEAM-DIMAP')

Apply TOPSAR Deburst...
Writing the product...
Product written


**talvez topophase removal seja só para interferogram e não DEM**

In [166]:
def topophase_removal(product):
    parameters = HashMap()
    print('Apply Topographical Phase Removal...')
    parameters.put("Orbit Interpolation Degree", 3)
    parameters.put("Digital Elevation Model", "Copernicus 30m Global DEM")
    parameters.put("Tile Extension[%]", 100)
    parameters.put("Output topographic phase band", True)
    parameters.put("Output elevation band", False)
    output = GPF.createProduct("TopoPhaseRemoval", parameters, product)
    return output

In [168]:
#topo_phase_removal = topophase_removal(deburst)

In [171]:
def goldstein_phase_filtering(product):
    parameters = HashMap()
    print('Apply Goldstein Phase Filtering...')
    parameters.put('alpha', 1.0)
    parameters.put('FFTSizeString', '64')
    parameters.put('windowSizeString', '3')
    parameters.put('useCoherenceMask', False)
    parameters.put('coherenceThreshold', 0.2)  
    output = GPF.createProduct("GoldsteinPhaseFiltering", parameters, product)
    return output

In [173]:
goldstein = goldstein_phase_filtering(read_product(r'_results\deburst.dim'))
write(deburst, r'_results\goldstein', 'BEAM-DIMAP')

Apply Goldstein Phase Filtering...
Writing the product...
Product written


In [123]:
# Reload it
reloaded_product = ProductIO.readProduct(r'_results\goldstein.dim')

# Now try reading from the reloaded product
phase_band = reloaded_product.getBand('Phase_ifg_IW2_VH_08Sep2021_19Dec2021')
phase_data = np.zeros(1000 * 1000, dtype=np.float32)
phase_band.readPixels(11749, 6289, 1000, 1000, phase_data)

print("After reloading:")
print(f"  Mean: {np.mean(phase_data):.6f}")
print(f"  Std: {np.std(phase_data):.6f}")
print(f"  Non-zero pixels: {np.count_nonzero(phase_data)}")

After reloading:
  Mean: 0.000000
  Std: 0.000000
  Non-zero pixels: 0


**talvez multilooking seja só para interferogram e não DEM**

In [43]:
def multilook(product):
    parameters = HashMap()
    print('Multilooking...')
    #parameters.put("nRgLooks", 6)
    parameters.put("outputIntensity", True)
    parameters.put('grSquarePixel',True)
    output = GPF.createProduct("Multilook", parameters, product)
    return output

In [44]:
#multilooking = multilook(goldstein)

In [45]:
def snaphu_export(product, export_folder_path):
    parameters = HashMap()
    print('Exporting SNAPHU...')
    parameters.put('targetFolder', export_folder_path)  
    output = GPF.createProduct('SnaphuExport', parameters, product)
    write(output, export_folder_path, 'Snaphu')
    return output

In [ ]:
snaphu = snaphu_export(goldstein, '_results\snaphu_export4')

Exporting SNAPHU...
Writing the product...
